<a href="https://colab.research.google.com/github/aneeq-shaffy/SE4050-Deep-Learning/blob/main/XLM_RoBERTa_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Preprocessing**

### Importing necessary libraries

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import (
    XLMRobertaTokenizer,
    XLMRobertaForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
import torch
import warnings
warnings.filterwarnings('ignore')
import json
import requests

Read Dataset

In [5]:
def read_lk_news_automatically(year='2021', num_articles=99999):
    # Get folder list from GitHub
    api_url = f"https://api.github.com/repos/nuuuwan/lk_news/contents/data/lk_news/2020s/{year}?ref=data"
    response = requests.get(api_url)

    if response.status_code != 200:
        print("Error getting folder list")
        return None

    folders = response.json()
    print(f"Found {len(folders)} articles in {year}")

    all_articles = []
    base_url = f"https://raw.githubusercontent.com/nuuuwan/lk_news/data/data/lk_news/2020s/{year}"

    # Download each article
    for i, folder in enumerate(folders[:num_articles]):
        if folder['type'] == 'dir':
            folder_name = folder['name']

            try:
                # Get files
                metadata = requests.get(f"{base_url}/{folder_name}/doc.json").json()
                text = requests.get(f"{base_url}/{folder_name}/doc.txt").text

                # Store article
                all_articles.append({
                    'id': metadata.get('doc_id', ''),
                    'text': text,
                    'title': metadata.get('description', ''),
                    'language': metadata.get('lang', ''),
                    'date': metadata.get('date_str', '')
                })

                if (i+1) % 10 == 0:
                    print(f"Loaded {i+1} articles...")

            except:
                print(f"Failed: {folder_name}")

    return pd.DataFrame(all_articles)

In [6]:
def get_all_news(years=['2021', '2022', '2023', '2024', '2025'], articles_per_year=50):
    all_dfs = []

    for year in years:
        print(f"\nGetting {year}...")
        df_year = read_lk_news_automatically(year, articles_per_year)
        if df_year is not None:
            all_dfs.append(df_year)

    final_df = pd.concat(all_dfs, ignore_index=True)
    print(f"\n✅ Total: {len(final_df)} articles")
    return final_df

In [8]:
# Get the data
df = get_all_news(years=['2021', '2022', '2023', '2024', '2025'], articles_per_year=50)


Getting 2021...
Found 4 articles in 2021

Getting 2022...
Found 19 articles in 2022
Loaded 10 articles...

Getting 2023...
Found 173 articles in 2023
Loaded 10 articles...
Loaded 20 articles...
Loaded 30 articles...
Loaded 40 articles...
Loaded 50 articles...

Getting 2024...
Found 1000 articles in 2024
Loaded 10 articles...
Loaded 20 articles...
Loaded 30 articles...
Loaded 40 articles...
Loaded 50 articles...

Getting 2025...
Found 1000 articles in 2025
Loaded 10 articles...
Loaded 20 articles...
Loaded 30 articles...
Loaded 40 articles...
Loaded 50 articles...

✅ Total: 173 articles


In [9]:
# See what you got
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst article:")
print(df.iloc[0])

Shape: (173, 5)
Columns: ['id', 'text', 'title', 'language', 'date']

First article:
id                                  2021-09-12-adalk-095b8035
text        නොවැම්බර් මස සිට WhatsApp අත්හිටුවන ජංගම දුරකථ...
title       නොවැම්බර් මස සිට WhatsApp අත්හිටුවන ජංගම දුරකථ...
language                                                   si
date                                               2021-09-12
Name: 0, dtype: object


In [10]:
# Save to CSV
df.to_csv('lk_news_data.csv', index=False)
print("💾 Saved to lk_news_data.csv")

💾 Saved to lk_news_data.csv


In [11]:
df=pd.read_csv('lk_news_data.csv')

In [12]:
df.head()

,id,text,title,language,date
0,2021-09-12-adalk-095b8035,නොවැම්බර් මස සිට WhatsApp අත්හිටුවන ජංගම දුරකථ...,නොවැම්බර් මස සිට WhatsApp අත්හිටුවන ජංගම දුරකථ...,si,2021-09-12
1,2021-09-22-adalk-28eb5276,ගෙදරදීම පොල්තෙල් සිඳින අත්යන්ත්‍රයක්\n\nවස වි...,ගෙදරදීම පොල්තෙල් සිඳින අත්යන්ත්‍රයක්,si,2021-09-22
2,2021-10-04-adalk-f6f01c09,සමාජ මාධ්‍ය බි﻿ඳ වැටීම යථා තත්ත්වයට\n\n(UPDATE...,සමාජ මාධ්‍ය බි﻿ඳ වැටීම යථා තත්ත්වයට,si,2021-10-04
3,2021-10-14-adalk-4868ea0d,Number Portability සේවාව සදහා නීතිමය අනුමැතිය\...,Number Portability සේවාව සදහා නීතිමය අනුමැතිය,si,2021-10-14
4,2022-01-31-adalk-7f0da7a4,යුද හමුදාවෙන් තවත් සුපිරි වැඩක්\n\nශ්‍රී ලංකා ...,යුද හමුදාවෙන් තවත් සුපිරි වැඩක්,si,2022-01-31


In [13]:
df.tail()

,id,text,title,language,date
168,2025-01-01-adaderanasinhalalk-fbe67685,නව වසරට ජනපතිගෙන් සුබ පැතුම්\n\n2024 වසරේ ලබාග...,නව වසරට ජනපතිගෙන් සුබ පැතුම්,si,2025-01-01
169,2025-01-01-adaderanasinhalalk-ff3a3a00,මාතර බන්ධනාගාරයේ බෝ ගසක අත්තක් කඩාවැටී අයෙක් ම...,මාතර බන්ධනාගාරයේ බෝ ගසක අත්තක් කඩාවැටී අයෙක් මරුට,si,2025-01-01
170,2025-01-01-adalk-03d0dbe1,මාඳු ගඟ අභය භූමිය බලන්න එන අයගෙන් මුදල් අරගෙන ...,මාඳු ගඟ අභය භූමිය බලන්න එන අයගෙන් මුදල් අරගෙන නෑ,si,2025-01-01
171,2025-01-01-adalk-056eb2bd,ඉන්දු ලංකා මගී නැව් සේවාව යළි හෙට සිට ඇරඹෙයි\n...,ඉන්දු ලංකා මගී නැව් සේවාව යළි හෙට සිට ඇරඹෙයි,si,2025-01-01
172,2025-01-01-adalk-05827ff1,උදලු පහරින් බිරිඳ මරුට\n\nකුලියාපිටිය පරගහරුප්...,උදලු පහරින් බිරිඳ මරුට,si,2025-01-01


Sanity Check of the data

In [14]:
df.shape

(173, 5)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        173 non-null    object
 1   text      173 non-null    object
 2   title     173 non-null    object
 3   language  173 non-null    object
 4   date      173 non-null    object
dtypes: object(5)
memory usage: 6.9+ KB


In [16]:
df.isnull().sum()

,0
id,0
text,0
title,0
language,0
date,0


In [18]:
df.duplicated().sum()

np.int64(0)

In [20]:
df['date'] = pd.to_datetime(df['date'])
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        173 non-null    object        
 1   text      173 non-null    object        
 2   title     173 non-null    object        
 3   language  173 non-null    object        
 4   date      173 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 6.9+ KB


None

In [21]:
for col in df.select_dtypes(include='object').columns:
    print(f"Column: {col}")
    print(df[col].value_counts().head()) # Display top 5 value counts
    print(f"Number of unique values: {df[col].nunique()}")
    print("-" * 30)

Column: id
id
2021-09-12-adalk-095b8035    1
2021-09-22-adalk-28eb5276    1
2021-10-04-adalk-f6f01c09    1
2021-10-14-adalk-4868ea0d    1
2022-01-31-adalk-7f0da7a4    1
Name: count, dtype: int64
Number of unique values: 173
------------------------------
Column: text
text
නොවැම්බර් මස සිට WhatsApp අත්හිටුවන ජංගම දුරකථන ලයිස්තුව\n\nනොවැම්බර් මස සිට පැරණි ස්මාට් ජංගම දුරකථන මාලිදි විශාල සංඛ්‍යාවකට WhatsApp සමාජ මාධ්‍ය යෙදුම ලබාගත නොහැකි බව එම සමාගම දැනුම් දී ඇත.\n\nඇපල් අයිෆෝන් වල මෙහෙයුම් පද්ධතිය IOS 9 ට සමාන හෝ අඩු සහ ඇන්ඩ්‍රොයිඩ් 4.0.3 ට සමාන හෝ අඩු මෙහෙයුම් පද්ධතියක් ඇති ජංගම දුරකථනවලට මෙලෙස නොවැම්බර් මාසයේ සිට WhatsApp භාවිතා කිරීමේ හැකියාව නොලැබී යනු ඇ​ත.\n\n* Samsung: Galaxy Trend II, Galaxy SII, Samsung Galaxy Trend Lite, Galaxy S3 mini, Galaxy Ace 2, Galaxy Xcover 2, and Galaxy Core\n\n* LG: LG Lucid 2, Optimus F7, Optimus F5, Optimus L3 II Dual, Optimus F5, Optimus L5, Optimus L5 II, Optimus L5 Dual, Optimus L3 II, Optimus L7, Optimus L7 II Dual, Optimus L7 II, Optimus F6, Enac